In [1]:
# libs
import datetime
import ctypes
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import logging
import os
from time import sleep
from tkinter import Tk, Label, Button

current_dir = os.getcwd()
input_csv = os.path.join(current_dir,'input.csv')
log_file = os.path.join(current_dir, "consulta_cnpj.log")
with open(input_csv) as f:
    # Lendo todo o arquivo na variável cnpjs
    cnpjs = f.readlines()[1:]
    # Removendo caracteres não numéricos
    cnpjs = [''.join(filter(str.isdigit, cnpj)) for cnpj in cnpjs]
    # Removendo duplicados
    cnpjs = list(set(cnpjs))
print(cnpjs)

['00957404000178', '03389126000198', '38486817000194', '28799908000126', '17319831000123', '17444779000137', '32384286000142', '17503475000101', '40004800000103', '06151940000109', '17486275000180', '17387481000132', '99999997984356', '17498205000141', '99999999992022', '19198118000102', '16636540000104', '16866394000103', '16907746000113', '18715516000188', '33396928000196', '99999997984275', '19843929000100', '16789398000127', '65172579000115', '18715532000170', '13235618000182', '19701818000160', '20971057000145', '17255670000151', '03919139000121', '29768219000117', '99999999992103', '05585681000110', '16745465000101', '09104426000160', '12252931000166', '15438067000180', '17281106000103', '05465167000141', '21154554000113', '03133408000120', '17516113000147', '22261473000185', '17217332000125', '19296342000129', '21727508000166', '26388330000190', '21229281000129', '18715615000160', '18746164000128', '18715581000103', '32384344000138', '07256298000144', '10398157000170', '21949888

In [1]:
def input_ok ():
    # Criando a janela principal do Tkinter
    root = Tk()

    # Adicionando uma Label para exibir a mensagem ao usuário
    label = Label(root, text="Resolva o captcha manualmente e clique em Continuar.")
    label.configure(font={'size': 20})
    label.pack()

    # Função para continuar a execução do script quando o botão for clicado
    def on_button_click():
        # Fechando a janela do Tkinter
        root.destroy()

    # Adicionando um botão para permitir que o usuário continue a execução do script
    button = Button(root, text="Continuar", command=on_button_click)
    button.pack()

    # Exibindo a janela do Tkinter
    root.mainloop()

In [4]:
import pytesseract
caminho = tessdata_dir_config = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = caminho
tessdata_dir_config = '--tessdata-dir "C:\\Program Files\\Tesseract-OCR\\tessdata"'
captcha_text = pytesseract.image_to_string('captcha.png')
captcha_text

''

In [9]:

# gerando log
logging.basicConfig(level=logging.INFO, filename=log_file,
                    encoding='utf-8', format="%(asctime)s - %(levelname)s - %(message)s")

# Pegando a resolução da tela

user32 = ctypes.windll.user32
resolucao = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)

# Omitindo o navegador na execução
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation","enable-logging"])
chrome_options.add_argument(f'--window-size={resolucao[0]},{resolucao[1]}')
chrome_options.add_argument("start-maximized")
chrome_options.add_argument('--enable-cookies')


# Parâmetros do navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
for cnpj in cnpjs:
    driver.get(f"https://solucoes.receita.fazenda.gov.br/Servicos/cnpjreva/Cnpjreva_Solicitacao.asp")
    driver.implicitly_wait(15)
    driver.maximize_window()
    actions = ActionChains(driver, duration=100)
    
    # Inputa o cnpj
    driver.find_element(By.NAME,"cnpj").send_keys(cnpj)

    # Tenta resolver o captcha
    # Mudando para o iframe
    iframe = driver.find_element(By.XPATH,'/html/body/div/div[1]/div[1]/div/div/form/div[1]/div[2]/div/iframe')
    driver.switch_to.frame(iframe)

    # Clicando no captcha
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "checkbox"))).click()

    # Pausando a execução do script até que o usuário pressione Enter
    #input("Pressione `Enter` depois de resolver o captcha manualmente.")
    input_ok()

    # Mudando de volta para a janela principal
    driver.switch_to.default_content()

    # Clicando no  botão consultar
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div/div[1]/div[1]/div/div/form/div[3]/div/button[1]'))).click()
    
    break
sleep(5)
#driver.close()

In [2]:
import asyncio
from playwright.async_api import async_playwright
async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context()
        page = await context.new_page()

        for cnpj in cnpjs:
            await page.goto("https://solucoes.receita.fazenda.gov.br/Servicos/cnpjreva/Cnpjreva_Solicitacao.asp")
            await page.wait_for_selector('input[name="cnpj"]')
            await page.fill('input[name="cnpj"]', cnpj)

            # Tenta resolver o captcha
            iframe = await page.wait_for_selector('iframe')
            frame = await iframe.content_frame()
            await frame.wait_for_selector('#checkbox')
            await frame.click('#checkbox')

            # Aguarde o usuário resolver o captcha manualmente (pressionar Enter)
            await input("Pressione `Enter` depois de resolver o captcha manualmente.")

            # Clique no botão consultar
            await page.click('button:has-text("Consultar")')
            
            
            sleep(5)
            break

        await context.close()
        await browser.close()

# Executa o loop assíncrono
asyncio.get_event_loop().run_until_complete(main())

RuntimeError: This event loop is already running